In [1]:
import mlflow.pytorch
import torch
import numpy as np
import sys
import os

sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../src/models'))

from src.models.api import evaluate
from src.data.utils import load
from src.models.conve import ConveEvaluationModel

In [3]:
os.chdir('/mnt/c/Users/Duong Hai Dang/Developer/practical-big-data-science-adl-ai/')
maps, data = load()
train_triples, valid_triples, test_triples = data.values()

In [5]:
sftp_path = 'sftp://sftpuser@10.195.1.54/sftpuser/mlruns/2/0278ec00cc7b47eda553db7c4f66120e/artifacts/models/conve-model-43'
conve_model = mlflow.pytorch.load_model(sftp_path, map_location='cpu')

device = torch.device('cpu')
conve_model.device = device

all_positive_triples = np.concatenate([train_triples, valid_triples, test_triples], axis=0)
all_positive_triples_tensor = torch.tensor(all_positive_triples,
                                    dtype=torch.long,
                                    device=device)

all_entities_tensor = torch.arange(conve_model.num_entities,
                            dtype=torch.long,
                            device=device)

model = ConveEvaluationModel(conve_model, all_positive_triples_tensor, all_entities_tensor, device='cpu')

In [12]:
results = conve_model.evaluate(train_triples=np.concatenate([train_triples, valid_triples], axis=0), 
                               test_triples=test_triples)

Corrupting triples: 100%|██████████| 20466/20466 [07:00<00:00, 33.36it/s]


In [8]:
import tempfile
with tempfile.TemporaryDirectory() as tmpdir:
        mlflow.set_tracking_uri('http://10.195.1.54')
        mlflow.set_experiment('conve-error-analysis')
        mlflow.start_run(run_name='updated_conve_evaluation')
        evalModel.generate_model_output(output_path=tmpdir,
                                        test_triples=test,
                                        evaluation_result=results)

        files = os.listdir(tmpdir)

        for f in files:
            mlflow.log_artifact(os.path.join(tmpdir, f))

        mlflow.end_run()

In [13]:
results

{'hits_at_1': 0.22073194566598262,
 'hits_at_3': 0.33636274797224663,
 'hits_at_10': 0.47828105150004885,
 'mean_rank': 292.32634613505326,
 'mean_reciprocal_rank': 0.30646086510207704}

In [14]:
unfiltered = conve_model.evaluate(train_triples=np.concatenate([train_triples, valid_triples], axis=0), 
                               test_triples=test_triples, filter_fn=False)

Corrupting triples: 100%|██████████| 20466/20466 [09:42<00:00, 35.14it/s]


In [15]:
unfiltered

{'hits_at_1': 0.10913221929053063,
 'hits_at_3': 0.17985927880386984,
 'hits_at_10': 0.3089025701162904,
 'mean_rank': 457.50090393823905,
 'mean_reciprocal_rank': 0.17469986985414263}